# Расчётно-графическая работа №2
#### Выполнил: Сухов Николай Михайлович
#### Поток: 24.4
#### Вариант: 3 2

In [1]:
import numpy as np
from scipy.stats import f, norm
np.random.seed(17)

# Задание 1

Доверительный интервал уровня 1-$\alpha$ для параметра $\tau$

По теореме Фишера для $X_1,...,X_n ∼ 𝑁 (𝜇, 𝜎)$ верно такое свойство:
$$\frac{nS_*^2}{\sigma^2}=\frac{(n-1)S^2}{\sigma^2} \sim \chi^2 (n-1),$$
где $\chi^2$ - распределение Хи-квадрат. Так же распределение Фишера:
$$F(n_2,n_1)=\frac{\chi^2 (n_2)/n_2}{\chi^2 (n_1)/n_1}$$
Отсюда получаем:
$$\frac{n_1(n_2-1)S^2_\ast(X_1)}{n_2(n_1-1)S^2_\ast(X_2)} \cdot \frac{\sigma_2^2}{\sigma_1^2}=F(n_1, n_2)$$
Доверительный интервал:
$$P \left(\frac{n_2(n_1-1)S^2_\ast(X_2)}{n_1(n_2-1)S^1_\ast(X_1)} \tau \leq q_{1-\alpha/2}\right) = 1 - \alpha$$

$$P \left(\frac{n_1(n_2-1)S^2_\ast(X_1)}{n_2(n_1-1)S^2_\ast(X_2)} q_{\alpha/2} \leq  \tau \leq \frac{n_1(n_2-1)S^2_\ast(X_1)}{n_2(n_1-1)S^2_\ast(X_2)} q_{1-\alpha/2}\right) = 1 - \alpha$$

In [2]:
mu_1 = mu_2 = 0
sigma_sq_1, sigma_sq_2 = 2, 1
tau = sigma_sq_1 / sigma_sq_2
alpha_by_2 = 0.025
n1_25 = n2_25 = 25
n1_10000 = n2_10000 = 10000
repeats = 1000

In [3]:
def count_intrvals(mu_1, mu_2, n1, n2, sigma_sq_1, sigma_sq_2, alpha_by_2=0.025):
    x1 = np.random.normal(mu_1, np.sqrt(sigma_sq_1), n1)
    x2 = np.random.normal(mu_2, np.sqrt(sigma_sq_2), n2)
    s1_sq = np.var(x1, ddof=1)
    s2_sq = np.var(x2, ddof=1)
    f_lower = f.ppf(alpha_by_2, dfn=n1 - 1, dfd=n2 - 1)
    f_upper = f.ppf(1 - alpha_by_2, dfn=n1 - 1, dfd=n2 - 1)

    c = (n1 * (n2 - 1) * s1_sq) / (n2 * (n1 - 1) * s2_sq)
    lower = c * f_lower
    upper = c * f_upper
    
    return lower, upper

In [4]:
covered_25, covered_10000 = 0, 0

for _ in range(repeats):
    lower_25, upper_25 = count_intrvals(mu_1, mu_2, n1_25, n2_25, sigma_sq_1, sigma_sq_2)

    if lower_25 <= tau <= upper_25:
        covered_25 += 1

    lower_10000, upper_10000 = count_intrvals(mu_1, mu_2, n1_10000, n2_10000, sigma_sq_1, sigma_sq_2)

    if lower_10000 <= tau <= upper_10000:
        covered_10000 += 1

print(f"Объём выборки 25: доверительный интервал покрыл истинное значение τ = {tau} в {covered_25} из {repeats} случаев.")
print(f"Объём выборки 10000: доверительный интервал покрыл истинное значение τ = {tau} в {covered_10000} из {repeats} случаев.")


Объём выборки 25: доверительный интервал покрыл истинное значение τ = 2.0 в 959 из 1000 случаев.
Объём выборки 10000: доверительный интервал покрыл истинное значение τ = 2.0 в 954 из 1000 случаев.


Как мы видим, при большем объёме выборки, покрытие не меняется сильно. Это связано с тем, что при росте выборки увеличивается точность и доверительные интервалы становятся уже, но покрытие остаётся прежним. Это согласуется с теорией.

# Задание 2

В распределении Лапласа математическое ожидание и медиана совпадает с параметром сдвига. Воспользуемся этим:
$$\text{E}X=\mu=\beta=2,\qquad \text{D}X=\frac{2}{\alpha^2}=\frac{2}{1}=2$$
Центральная предельная теорема утверждает, что сумма достаточно большого количества слабо зависимых случайных величин, имеющих примерно одинаковые масштабы, имеет распределение, близкое к нормальному. Тогда Квантиль уровня $\alpha$ стандартного нормального распределения:
$$Z=\sqrt{n} \frac{\overline{X}_n - \mu}{\sigma} \rightarrow N(0,1)$$
Доверительный интервал:
$$P(-Z_{1-\alpha/2} \le Z \le Z_{1 - \alpha/2}) = 1 - \alpha$$
$$P(-Z_{1-\alpha/2} \le \sqrt{n} \frac{\overline{X}_n - \mu}{\sigma} \le Z_{1 - \alpha/2}) = 1 - \alpha$$
$$P(-\frac{Z_{1 - \alpha/2}\sqrt{D}}{\sqrt{n}}+\overline{X}_n \le \mu \le  \frac{Z_{\alpha/2}\sqrt{D}}{\sqrt{n}}+\overline{X}_n) = 1 - \alpha$$

In [5]:
mu = 2
alpha = 1
sigma = np.sqrt(mu)
z = norm.ppf(1 - alpha_by_2)

In [6]:
def count_asymptotic_intervals(mu, n, sigma, z):
    sample = np.random.laplace(loc=mu, scale=1, size=n)
    sample_mean = np.mean(sample)
    margin = z * sigma / np.sqrt(n)
    lower = sample_mean - margin
    upper = sample_mean + margin
    return lower, upper

In [7]:
covered_25l = 0
covered_10000l = 0

for _ in range(repeats):
    lower_25, upper_25 = count_asymptotic_intervals(mu, n1_25, sigma, z)
    if lower_25 <= mu <= upper_25:
        covered_25l += 1

    lower_10000, upper_10000 = count_asymptotic_intervals(mu, n1_10000, sigma, z)
    if lower_10000 <= mu <= upper_10000:
        covered_10000l += 1

print(f"Объём выборки 25: доверительный интервал покрыл истинное значение μ = {mu} в {covered_25l} из {repeats} случаев.")
print(f"Объём выборки 10000: доверительный интервал покрыл истинное значение μ = {mu} в {covered_10000l} из {repeats} случаев.")

Объём выборки 25: доверительный интервал покрыл истинное значение μ = 2 в 947 из 1000 случаев.
Объём выборки 10000: доверительный интервал покрыл истинное значение μ = 2 в 961 из 1000 случаев.


Как мы видим, итоги второго эксперимента подтвержают первый.